In [1]:
pip install wget

  Stored in directory: C:\Users\Faizan\AppData\Local\pip\Cache\wheels\40\15\30\7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


In [0]:
import wget

In [5]:
wget.download('https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip')
wget.download('https://raw.githubusercontent.com/google-research/bert/master/modeling.py')
wget.download('https://raw.githubusercontent.com/google-research/bert/master/optimization.py')
wget.download('https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py') 
wget.download('https://raw.githubusercontent.com/google-research/bert/master/tokenization.py')

'tokenization.py'

In [6]:
import modeling
import optimization
import run_classifier
import tokenization

In [3]:
import pandas as pd
import zipfile

In [0]:
folder = 'model_folder'
with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall(folder)

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1v6nCMY9_ir5E1zqb1JNKO2_ZsbchIsNF'

In [13]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1v6nCMY9_ir5E1zqb1JNKO2_ZsbchIsNF


In [4]:
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('Partial Preprocessed articles.csv')  
data = pd.read_csv('Partial Preprocessed articles.csv')
# Dataset is now stored in a Pandas Dataframe

In [5]:
data.head()

,Unnamed: 0,claim,claimant,date,id,label,related_articles,articles_len,preprocessed_articles,all_preprocessed_articles
0,0,A line from George Orwell's novel 1984 predict...,NaN,2017-07-17,0,0,"[122094, 122580, 130685, 134765]",4,1984 george orwell predicted 2017 almost 70 y...,1984 george orwell predicted 2017 almost 70 y...
1,1,Maine legislature candidate Leslie Gibson insu...,NaN,2018-03-17,1,2,"[106868, 127320, 128060]",3,republican criticized parkland students drops ...,republican criticized parkland students drops ...
2,2,A 17-year-old girl named Alyssa Carson is bein...,NaN,2018-07-18,4,1,"[132130, 132132, 149722]",3,first person complete nasas passport program a...,first person complete nasas passport program a...
3,3,In 1988 author Roald Dahl penned an open lette...,NaN,2019-02-04,5,2,"[123254, 123418, 127464]",3,dangerous measles tweet 27 march 2013 many pa...,dangerous measles tweet 27 march 2013 many pa...
4,4,"When it comes to fighting terrorism, ""Another ...",Hillary Clinton,2016-03-22,6,2,"[41099, 89899, 72543, 82644, 95344, 88361]",6,remarks counterterrorism stanford university s...,remarks counterterrorism stanford university s...


In [6]:
df = pd.DataFrame()

In [7]:
df.loc[:,'Text'] = data.claim + data.preprocessed_articles 

In [8]:
df.loc[:, 'Label'] = data.label

In [9]:
from sklearn.preprocessing import LabelEncoder

In [20]:
df2 = pd.DataFrame()
df2["Text"] = df["Text"]
df2["Label"] = df["Label"]
df2 = df2
print(df2.head)
print(df2.shape)

<bound method NDFrame.head of                                                     Text  Label
0      A line from George Orwell's novel 1984 predict...      0
1      Maine legislature candidate Leslie Gibson insu...      2
2      A 17-year-old girl named Alyssa Carson is bein...      1
3      In 1988 author Roald Dahl penned an open lette...      2
4      When it comes to fighting terrorism, "Another ...      2
5      Rhode Island is "almost dead last" among North...      2
6      The poorest counties in the U.S. are in Appala...      1
7      Koch Industries paid the legal fees of George ...      0
8      "Minnesota, Michigan, Iowa already have 70 mph...      1
9      "FBI Uniform Crime Report for 2016 shows more ...      1
10      "Pelosi Sinks to New Low, Tells Dems: If You ...      0
11     Socialist teachers at South Charlotte Middle S...      1
12     Says that in the U.S. Capitol, "Stephen F. Aus...      1
13     NASA Has Just Confirmed Earth Has A New Moonan...      0
14     "We

In [21]:
df2['Label'].value_counts().sort_values(ascending=False) #.iplot(kind='bar', yTitle='Number of Complaints', 
                                                                #title='Number complaints in each product')

0    7408
1    6451
2    1696
Name: Label, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df2["Text"].values, df2["Label"].values, test_size=0.2, random_state=42)

In [0]:
def create_examples(lines, set_type, labels=None):
#Generate data for the BERT model
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = line
            label = str(label)
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    else:
        for line in lines:
            text_a = line
            label = '0'
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples

# Model Hyper Parameters
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 1e-5
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 50
# Model configs
SAVE_CHECKPOINTS_STEPS = 100000 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 100000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

label_list = [str(num) for num in range(8)]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(X_train, 'train', labels=y_train)

tpu_cluster_resolver = None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available 
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': 'model_folder/outputs', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service':

In [0]:
print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('>> Started training at {} '.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.datetime.now()))

Please wait...

INFO:tensorflow:Writing example 0 of 12444
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train
INFO:tensorflow:tokens: [CLS] claimed that “ because of obama ##care , premium ##s on everybody have gone up … whether you ’ re in an employer - based system or not . ” health insurance premium spin new analysis affordable care act prompt ##s republicans white house trade misleading claims laws [SEP]
INFO:tensorflow:input_ids: 101 3555 2008 1523 2138 1997 8112 16302 1010 12882 2015 2006 7955 2031 2908 2039 1529 3251 2017 1521 2128 1999 2019 11194 1011 2241 2291 2030 2025 1012 1524 2740 5427 12882 6714 2047 4106 15184 2729 2552 25732 2015 10643 2317 2160 3119 22369 4447 4277 102
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 0 (id = 0)
INFO:

In [0]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    print(params)
    batch_size = 500

    num_examples = len(features)

    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [0]:
predict_examples = create_examples(X_test, 'test')

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

INFO:tensorflow:Writing example 0 of 3111
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test
INFO:tensorflow:tokens: [CLS] a proposed ban on hollow - point bullets and bullets that expand upon impact " essentially ban ##s deer hunting . " wi ##sp ##oli ##tics com rep ke ##ssler statement l ##rb 08 ##9 ##4 2 19 2013 ##de ##ar colleagues receiving feedback constituents colleagues regarding circulation [SEP]
INFO:tensorflow:input_ids: 101 1037 3818 7221 2006 8892 1011 2391 10432 1998 10432 2008 7818 2588 4254 1000 7687 7221 2015 8448 5933 1012 1000 15536 13102 10893 14606 4012 16360 17710 23385 4861 1048 15185 5511 2683 2549 1016 2539 2286 3207 2906 8628 4909 12247 24355 8628 4953 9141 102
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 0 (id = 0)
INFO:

In [0]:
preds = []
for prediction in result:
      preds.append(np.argmax(prediction['probabilities']))

{}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 50)
INFO:tensorflow:  name = input_mask, shape = (?, 50)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 50)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/l

In [0]:
from sklearn.metrics import accuracy_score
print("Accuracy of BERT is:",accuracy_score(y_test,preds))

Accuracy of BERT is: 0.6187720990035358


In [0]:
from sklearn.metrics import classification_report

In [0]:
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.69      0.67      0.68      1494
           1       0.56      0.71      0.63      1268
           2       0.40      0.05      0.09       349

    accuracy                           0.62      3111
   macro avg       0.55      0.48      0.47      3111
weighted avg       0.60      0.62      0.59      3111

